In [ ]:
from p2pfl.node import Node
from p2pfl.learning.learner import MyNodeLearning
from p2pfl.agregator import FedAvg    
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=-1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía

        
test_convergence((2,2))

INFO:root:Nodo a la escucha en localhost 60801
INFO:root:Nodo a la escucha en localhost 60802
INFO:root:Conexión aceptada con ('localhost', 60802)
INFO:root:Broadcasting start learning...
INFO:root:Training...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epochs: -1
Train: 54000 Val:6000 Test:10000



  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [3]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.node import Node
import pytest
import time
import torch


n1 = Node("localhost")
n2 = Node("localhost")

n1.start()
n2.start()

def test_node_paring():

    # Conexión
    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) #Esperar por la asincronía
    assert len(n1.neightboors) == len(n2.neightboors)==1
    
   
    # Desconexión
    n2.neightboors[0].stop()
    time.sleep(0.1) #Esperar por la asincronía
    #assert len(n1.neightboors) == len(n2.neightboors)== 0
    
        
test_node_paring()

INFO:root:Nodo a la escucha en localhost 57234
INFO:root:Nodo a la escucha en localhost 57235
INFO:root:Conexión aceptada con ('localhost', 57234)
Se que me debo acabar
DEBUG:root:Closed connection: ('localhost', 57235)
DEBUG:root:Closed connection: ('localhost', 57234)


In [2]:
len(n1.neightboors)

0